# Unit 5: An Introduction to ML-Agents

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/thumbnail.png" alt="Thumbnail"/>

In this notebook, you'll learn about ML-Agents and train two agents:

- **SnowballTarget**: Learn to shoot snowballs onto spawning targets
- **Pyramids**: Press a button to spawn a pyramid, navigate to it, knock it over, and move to the gold brick at the top (uses curiosity)

After training, you'll be able to **watch your agents playing directly in your browser**.

📄 Course: https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

⬇️ Here is what **you will achieve at the end of this unit** ⬇️

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/pyramids.gif" alt="Pyramids"/>

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballtarget.gif" alt="SnowballTarget"/>

### 🎮 Environments:

- SnowballTarget
- Pyramids

### Before you start ✅

- **Enable Internet** in Kaggle session settings
- Add your HuggingFace write token as a Kaggle Secret named `HF_TOKEN`
  - Kaggle sidebar → Add-ons → Secrets → Add New Secret
  - Get your token at: https://huggingface.co/settings/tokens

## Install ML-Agents 🔽

We install ML-Agents in a Python 3.10 virtual environment to avoid compatibility issues with Kaggle's default Python 3.12.

In [2]:
%%capture
!add-apt-repository ppa:deadsnakes/ppa -y
!apt-get update -q
!apt-get install -y python3.10 python3.10-venv python3.10-distutils -q

In [4]:
%%capture
# Create the environment
!python3.10 -m venv /kaggle/working/mlagents-env
!/kaggle/working/mlagents-env/bin/pip install --upgrade pip -q

# Step A: Install PyTorch CPU first from the specific index
!/kaggle/working/mlagents-env/bin/pip install torch --index-url https://download.pytorch.org/whl/cpu

# Step B: Install mlagents and core dependencies from PyPI
!/kaggle/working/mlagents-env/bin/pip install \
    "mlagents==1.1.0" \
    "mlagents-envs==1.1.0" \
    "numpy==1.23.5" \
    "onnx==1.15.0" \
    "grpcio==1.48.2" \
    "protobuf==3.20.3"

In [5]:
# Verify installation
!/kaggle/working/mlagents-env/bin/mlagents-learn --help | head -50

usage: mlagents-learn [-h] [--env ENV_PATH] [--resume] [--deterministic]
                      [--force] [--run-id RUN_ID] [--initialize-from RUN_ID]
                      [--seed SEED] [--inference] [--base-port BASE_PORT]
                      [--num-envs NUM_ENVS] [--num-areas NUM_AREAS] [--debug]
                      [--env-args ...]
                      [--max-lifetime-restarts MAX_LIFETIME_RESTARTS]
                      [--restarts-rate-limit-n RESTARTS_RATE_LIMIT_N]
                      [--restarts-rate-limit-period-s RESTARTS_RATE_LIMIT_PERIOD_S]
                      [--torch] [--tensorflow] [--results-dir RESULTS_DIR]
                      [--timeout-wait TIMEOUT_WAIT] [--width WIDTH]
                      [--height HEIGHT] [--quality-level QUALITY_LEVEL]
                      [--time-scale TIME_SCALE]
                      [--target-frame-rate TARGET_FRAME_RATE]
                      [--capture-frame-rate CAPTURE_FRAME_RATE]
                      [--no-graphics] [--no-gr

## Download Environment Executables 🔽

In [6]:
!mkdir -p ./training-envs-executables/SnowballTarget
!mkdir -p ./training-envs-executables/Pyramids

In [7]:
%%capture
!pip install huggingface_hub gdown -q

In [8]:
# Download SnowballTarget
!wget "https://github.com/huggingface/Snowball-Target/raw/main/SnowballTarget.zip" -O ./training-envs-executables/SnowballTarget.zip

--2026-02-24 19:59:16--  https://github.com/huggingface/Snowball-Target/raw/main/SnowballTarget.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/huggingface/Snowball-Target/main/SnowballTarget.zip [following]
--2026-02-24 19:59:16--  https://media.githubusercontent.com/media/huggingface/Snowball-Target/main/SnowballTarget.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35134213 (34M) [application/zip]
Saving to: ‘./training-envs-executables/SnowballTarget.zip’

./training-envs-exe 100%[===================>]  33.51M  73.3MB/s    in 0.5s    

2026-02-24 19:59:17 (73.3 MB/s) 

In [9]:
%%capture
!unzip -d ./training-envs-executables/SnowballTarget/ ./training-envs-executables/SnowballTarget.zip

In [10]:
!chmod -R 755 ./training-envs-executables/SnowballTarget/

In [11]:
# Confirm the executable is present
!ls -la ./training-envs-executables/SnowballTarget/

total 12
drwxr-xr-x 3 root root 4096 Feb 24 19:59 .
drwxr-xr-x 4 root root 4096 Feb 24 19:59 ..
drwxr-xr-x 4 root root 4096 Jan  8  2023 SnowballTarget


In [12]:
!ls -R ./training-envs-executables/SnowballTarget/

./training-envs-executables/SnowballTarget/:
SnowballTarget

./training-envs-executables/SnowballTarget/SnowballTarget:
SnowballTarget_BurstDebugInformation_DoNotShip	SnowballTarget.x86_64
SnowballTarget_Data				UnityPlayer.so

./training-envs-executables/SnowballTarget/SnowballTarget/SnowballTarget_BurstDebugInformation_DoNotShip:
Data

./training-envs-executables/SnowballTarget/SnowballTarget/SnowballTarget_BurstDebugInformation_DoNotShip/Data:
Plugins

./training-envs-executables/SnowballTarget/SnowballTarget/SnowballTarget_BurstDebugInformation_DoNotShip/Data/Plugins:
lib_burst_generated.txt

./training-envs-executables/SnowballTarget/SnowballTarget/SnowballTarget_Data:
app.info		   Managed		  RuntimeInitializeOnLoads.json
boot.config		   MonoBleedingEdge	  ScriptingAssemblies.json
globalgamemanagers	   Plugins		  sharedassets0.assets
globalgamemanagers.assets  Resources		  sharedassets0.assets.resS
level0			   resources.assets	  StreamingAssets
level0.resS		   resources.assets.res

In [15]:
# Download Pyramids
!wget "https://huggingface.co/spaces/unity/ML-Agents-Pyramids/resolve/main/Pyramids.zip" -O ./training-envs-executables/Pyramids.zip

--2026-02-24 20:07:28--  https://huggingface.co/spaces/unity/ML-Agents-Pyramids/resolve/main/Pyramids.zip
Resolving huggingface.co (huggingface.co)... 3.170.185.25, 3.170.185.14, 3.170.185.35, ...
Connecting to huggingface.co (huggingface.co)|3.170.185.25|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/62b2c0284ea3eff279c966ab/2f754b6bf0b817493a77fafe582b7dde47fa25a476d3f8e68a24fdd4efcaa282?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Pyramids.zip%3B+filename%3D%22Pyramids.zip%22%3B&response-content-type=application%2Fzip&Expires=1771967248&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzcxOTY3MjQ4fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjJiMmMwMjg0ZWEzZWZmMjc5Yzk2NmFiLzJmNzU0YjZiZjBiODE3NDkzYTc3ZmFmZTU4MmI3ZGRlNDdmYTI1YTQ3NmQzZjhlNjhhMjRmZGQ0ZWZjYWEyODJcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoifV1

In [16]:
%%capture
!unzip -d ./training-envs-executables/Pyramids/ ./training-envs-executables/Pyramids.zip

In [17]:
!chmod -R 755 ./training-envs-executables/Pyramids/

In [18]:
# Confirm the executable is present
!ls -la ./training-envs-executables/Pyramids/

total 12
drwxr-xr-x 3 root root 4096 Feb 24 20:07 .
drwxr-xr-x 4 root root 4096 Feb 24 20:07 ..
drwxr-xr-x 4 root root 4096 Jun 23  2022 Pyramids


In [19]:
!ls -R ./training-envs-executables/Pyramids/

./training-envs-executables/Pyramids/:
Pyramids

./training-envs-executables/Pyramids/Pyramids:
Pyramids				  Pyramids_Data
Pyramids_BurstDebugInformation_DoNotShip  UnityPlayer.so

./training-envs-executables/Pyramids/Pyramids/Pyramids_BurstDebugInformation_DoNotShip:
Data

./training-envs-executables/Pyramids/Pyramids/Pyramids_BurstDebugInformation_DoNotShip/Data:
Plugins

./training-envs-executables/Pyramids/Pyramids/Pyramids_BurstDebugInformation_DoNotShip/Data/Plugins:
lib_burst_generated.txt

./training-envs-executables/Pyramids/Pyramids/Pyramids_Data:
app.info		   Managed	     resources.assets
boot.config		   ML-Agents	     RuntimeInitializeOnLoads.json
globalgamemanagers	   MonoBleedingEdge  ScriptingAssemblies.json
globalgamemanagers.assets  Plugins	     sharedassets0.assets
level0			   Resources	     sharedassets0.assets.resS

./training-envs-executables/Pyramids/Pyramids/Pyramids_Data/Managed:
Assembly-CSharp.dll
Assembly-CSharp.pdb
Google.Protobuf.dll
Google.Protobuf.pdb
Gr

## Part 1: SnowballTarget ❄️🎯

Train an agent to shoot snowballs at spawning targets.

### Create the SnowballTarget config file

In [20]:
import os

snowball_config = """
behaviors:
  SnowballTarget:
    trainer_type: ppo
    hyperparameters:
      batch_size: 128
      buffer_size: 2048
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: false
      hidden_units: 256
      num_layers: 2
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
    checkpoint_interval: 200000
    keep_checkpoints: 5
    max_steps: 1000000
    time_horizon: 64
    summary_freq: 10000
"""

os.makedirs("/kaggle/working/ml-agents/config/ppo", exist_ok=True)
with open("/kaggle/working/ml-agents/config/ppo/SnowballTarget.yaml", "w") as f:
    f.write(snowball_config)

print("✅ SnowballTarget config written to: /kaggle/working/ml-agents/config/ppo/SnowballTarget.yaml")

✅ SnowballTarget config written to: /kaggle/working/ml-agents/config/ppo/SnowballTarget.yaml


### Train SnowballTarget agent

Training takes ~30–45 minutes. The agent learns to aim and shoot snowballs at targets.

In [23]:
!find /kaggle/working/ -name "SnowballTarget.yaml"

/kaggle/working/ml-agents/config/ppo/SnowballTarget.yaml


In [24]:
!/kaggle/working/mlagents-env/bin/mlagents-learn \
    /kaggle/working/ml-agents/config/ppo/SnowballTarget.yaml \
    --env=/kaggle/working/training-envs-executables/SnowballTarget/SnowballTarget/SnowballTarget.x86_64 \
    --run-id="SnowballTarget1" \
    --no-graphics \
    --force


            ┐  ╖
        ╓╖╬│╡  ││╬╖╖
    ╓╖╬│││││┘  ╬│││││╬╖
 ╖╬│││││╬╜        ╙╬│││││╖╖                               ╗╗╗
 ╬╬╬╬╖││╦╖        ╖╬││╗╣╣╣╬      ╟╣╣╬    ╟╣╣╣             ╜╜╜  ╟╣╣
 ╬╬╬╬╬╬╬╬╖│╬╖╖╓╬╪│╓╣╣╣╣╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╒╣╣╖╗╣╣╣╗   ╣╣╣ ╣╣╣╣╣╣ ╟╣╣╖   ╣╣╣
 ╬╬╬╬┐  ╙╬╬╬╬│╓╣╣╣╝╜  ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╣╙ ╙╣╣╣  ╣╣╣ ╙╟╣╣╜╙  ╫╣╣  ╟╣╣
 ╬╬╬╬┐     ╙╬╬╣╣      ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣     ╣╣╣┌╣╣╜
 ╬╬╬╜       ╬╬╣╣      ╙╝╣╣╬      ╙╣╣╣╗╖╓╗╣╣╣╜ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣╦╓    ╣╣╣╣╣
 ╙   ╓╦╖    ╬╬╣╣   ╓╗╗╖            ╙╝╣╣╣╣╝╜   ╘╝╝╜   ╝╝╝  ╝╝╝   ╙╣╣╣    ╟╣╣╣
   ╩╬╬╬╬╬╬╦╦╬╬╣╣╗╣╣╣╣╣╣╣╝                                             ╫╣╣╣╣
      ╙╬╬╬╬╬╬╬╣╣╣╣╣╣╝╜
          ╙╬╬╬╣╣╣╜
             ╙
        
 Version information:
  ml-agents: 1.1.0,
  ml-agents-envs: 1.1.0,
  Communicator API: 1.5.0,
  PyTorch: 2.10.0+cpu
[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
[INFO] Connected new brain: SnowballTarget?team=0
[INF

In [31]:
!/kaggle/working/mlagents-env/bin/pip install "torch==2.0.1" --index-url https://download.pytorch.org/whl/cpu --force-reinstall
!/kaggle/working/mlagents-env/bin/pip install onnxscript onnx==1.15.0

Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.0.1%2Bcpu-cp310-cp310-linux_x86_64.whl (195.4 MB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached https://download.pytorch.org/whl/MarkupSafe-3.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached filelock-3.20.0-py3-none-any.whl (16 kB)
Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached https://download.pytorch.org/whl/MarkupSafe-3.0.2-cp310-cp310-manylinux_2_17_x8

In [32]:
!/kaggle/working/mlagents-env/bin/mlagents-learn \
    /kaggle/working/ml-agents/config/ppo/SnowballTarget.yaml \
    --env=/kaggle/working/training-envs-executables/SnowballTarget/SnowballTarget/SnowballTarget.x86_64 \
    --run-id="SnowballTarget1" \
    --no-graphics \
    --resume


            ┐  ╖
        ╓╖╬│╡  ││╬╖╖
    ╓╖╬│││││┘  ╬│││││╬╖
 ╖╬│││││╬╜        ╙╬│││││╖╖                               ╗╗╗
 ╬╬╬╬╖││╦╖        ╖╬││╗╣╣╣╬      ╟╣╣╬    ╟╣╣╣             ╜╜╜  ╟╣╣
 ╬╬╬╬╬╬╬╬╖│╬╖╖╓╬╪│╓╣╣╣╣╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╒╣╣╖╗╣╣╣╗   ╣╣╣ ╣╣╣╣╣╣ ╟╣╣╖   ╣╣╣
 ╬╬╬╬┐  ╙╬╬╬╬│╓╣╣╣╝╜  ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╣╙ ╙╣╣╣  ╣╣╣ ╙╟╣╣╜╙  ╫╣╣  ╟╣╣
 ╬╬╬╬┐     ╙╬╬╣╣      ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣     ╣╣╣┌╣╣╜
 ╬╬╬╜       ╬╬╣╣      ╙╝╣╣╬      ╙╣╣╣╗╖╓╗╣╣╣╜ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣╦╓    ╣╣╣╣╣
 ╙   ╓╦╖    ╬╬╣╣   ╓╗╗╖            ╙╝╣╣╣╣╝╜   ╘╝╝╜   ╝╝╝  ╝╝╝   ╙╣╣╣    ╟╣╣╣
   ╩╬╬╬╬╬╬╦╦╬╬╣╣╗╣╣╣╣╣╣╣╝                                             ╫╣╣╣╣
      ╙╬╬╬╬╬╬╬╣╣╣╣╣╣╝╜
          ╙╬╬╬╣╣╣╜
             ╙
        
 Version information:
  ml-agents: 1.1.0,
  ml-agents-envs: 1.1.0,
  Communicator API: 1.5.0,
  PyTorch: 2.0.1+cpu
[WARNING] PyTorch checkpoint was saved with a different version of PyTorch. Model may not resume properly.
[INFO] Connected to Unity environment with package 

## Part 2: Pyramids 🏛️

Train an agent to press a button, navigate to a pyramid, knock it over, and reach the gold brick using **curiosity-driven exploration**.

### Create the Pyramids config file (with curiosity)

In [39]:
pyramids_config = """
behaviors:
  Pyramids:
    trainer_type: ppo
    hyperparameters:
      batch_size: 128
      buffer_size: 2048
      learning_rate: 0.0003
      beta: 0.01
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: false
      hidden_units: 256
      num_layers: 2
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
      curiosity:
        gamma: 0.99
        strength: 0.02
        encoding_size: 256
        learning_rate: 0.0003
    checkpoint_interval: 200000
    keep_checkpoints: 5
    max_steps: 1500000
    time_horizon: 128
    summary_freq: 10000
"""

with open("/kaggle/working/ml-agents/config/ppo/Pyramids.yaml", "w") as f:
    f.write(pyramids_config)

print("✅ Pyramids config written (with curiosity reward) to: /kaggle/working/ml-agents/config/ppo/Pyramids.yaml")

✅ Pyramids config written (with curiosity reward) to: /kaggle/working/ml-agents/config/ppo/Pyramids.yaml


### Train Pyramids agent

Training takes ~45–60 minutes. Curiosity helps the agent explore and discover the pyramid task.

In [46]:
!/kaggle/working/mlagents-env/bin/mlagents-learn \
    /kaggle/working/ml-agents/config/ppo/Pyramids.yaml \
    --env=./training-envs-executables/Pyramids/Pyramids/Pyramids \
    --run-id="Pyramids1" \
    --no-graphics \
    --force

[WARNING] 'encoding_size' was deprecated for RewardSignals. Please use network_settings.

            ┐  ╖
        ╓╖╬│╡  ││╬╖╖
    ╓╖╬│││││┘  ╬│││││╬╖
 ╖╬│││││╬╜        ╙╬│││││╖╖                               ╗╗╗
 ╬╬╬╬╖││╦╖        ╖╬││╗╣╣╣╬      ╟╣╣╬    ╟╣╣╣             ╜╜╜  ╟╣╣
 ╬╬╬╬╬╬╬╬╖│╬╖╖╓╬╪│╓╣╣╣╣╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╒╣╣╖╗╣╣╣╗   ╣╣╣ ╣╣╣╣╣╣ ╟╣╣╖   ╣╣╣
 ╬╬╬╬┐  ╙╬╬╬╬│╓╣╣╣╝╜  ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╣╙ ╙╣╣╣  ╣╣╣ ╙╟╣╣╜╙  ╫╣╣  ╟╣╣
 ╬╬╬╬┐     ╙╬╬╣╣      ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣     ╣╣╣┌╣╣╜
 ╬╬╬╜       ╬╬╣╣      ╙╝╣╣╬      ╙╣╣╣╗╖╓╗╣╣╣╜ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣╦╓    ╣╣╣╣╣
 ╙   ╓╦╖    ╬╬╣╣   ╓╗╗╖            ╙╝╣╣╣╣╝╜   ╘╝╝╜   ╝╝╝  ╝╝╝   ╙╣╣╣    ╟╣╣╣
   ╩╬╬╬╬╬╬╦╦╬╬╣╣╗╣╣╣╣╣╣╣╝                                             ╫╣╣╣╣
      ╙╬╬╬╬╬╬╬╣╣╣╣╣╣╝╜
          ╙╬╬╬╣╣╣╜
             ╙
        
 Version information:
  ml-agents: 1.1.0,
  ml-agents-envs: 1.1.0,
  Communicator API: 1.5.0,
  PyTorch: 2.0.1+cpu
[INFO] Connected to Unity environment with package version 2.2.1-exp.

## Push agents to the 🤗 Hub

We use **Kaggle Secrets** for the HuggingFace token.

In [34]:
from huggingface_hub import notebook_login
# notebook_login()

In [35]:
!git config --global credential.helper store

In [36]:
import os
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

login(token=hf_token, add_to_git_credential=True)

api = HfApi()
user_info = api.whoami()
print(f"✅ Logged in as: {user_info['name']}")
print(f"✅ Token has Write access: {user_info['auth']['type'] == 'WRITE'}")

✅ Logged in as: Chiz
✅ Token has Write access: False


### Push SnowballTarget

**Edit `--repo-id` to use your HuggingFace username**

In [38]:
!/kaggle/working/mlagents-env/bin/mlagents-push-to-hf \
    --run-id="SnowballTarget1" \
    --local-dir="./results/SnowballTarget1" \
    --repo-id="Chiz/ppo-SnowballTarget" \
    --commit-message="Unit 5 SnowballTarget"

[INFO] This function will create a model card and upload your SnowballTarget1 into HuggingFace Hub. This is a work in progress: If you encounter a bug, please send open an issue
[INFO] Pushing repo SnowballTarget1 to the Hugging Face Hub
Processing Files (0 / 0)      : |                  |  0.00B /  0.00B            
New Data Upload               : |                  |  0.00B /  0.00B            

  ...owballTarget-1000696.onnx:   2%|▎             | 13.6kB /  651kB            


  ...SnowballTarget-1000696.pt:   2%|▎             | 80.7kB / 3.85MB            



  ...nowballTarget-399968.onnx:   2%|▎             | 13.6kB /  651kB            




  .../SnowballTarget-399968.pt:   2%|▎             | 80.7kB / 3.85MB            





  ...nowballTarget-599976.onnx:   2%|▎             | 13.6kB /  651kB            






  .../SnowballTarget-599976.pt:   2%|▎             | 80.7kB / 3.85MB            







  ...nowballTarget-799984.onnx:   2%|▎             | 13.6kB /  651kB            







### Push Pyramids

**Edit `--repo-id` to use your HuggingFace username**

In [47]:
!/kaggle/working/mlagents-env/bin/mlagents-push-to-hf \
    --run-id="Pyramids1" \
    --local-dir="./results/Pyramids1" \
    --repo-id="Chiz/ppo-Pyramids" \
    --commit-message="Unit 5 Pyramids with curiosity"

[INFO] This function will create a model card and upload your Pyramids1 into HuggingFace Hub. This is a work in progress: If you encounter a bug, please send open an issue
[INFO] Pushing repo Pyramids1 to the Hugging Face Hub
Processing Files (0 / 0)      : |                  |  0.00B /  0.00B            
New Data Upload               : |                  |  0.00B /  0.00B            

  ...ids/Pyramids-1199919.onnx:   2%|▎             | 8.73kB /  451kB            


  ...amids/Pyramids-1199919.pt:   2%|▎             | 81.2kB / 4.19MB            



  ...ids/Pyramids-1399964.onnx:   2%|▎             | 8.73kB /  451kB            




  ...amids/Pyramids-1399964.pt:   2%|▎             | 81.2kB / 4.19MB            





  ...ids/Pyramids-1500031.onnx:   2%|▎             | 8.73kB /  451kB            






  ...amids/Pyramids-1500031.pt:   2%|▎             | 81.2kB / 4.19MB            







  ...mids/Pyramids-799915.onnx:   2%|▎             | 8.73kB /  451kB            








  ...rami

## Congrats on finishing Unit 5! 🎉

You can now play with your agents in the browser:

- SnowballTarget: https://huggingface.co/spaces/ThomasSimonini/SnowballTarget
- Pyramids: https://huggingface.co/spaces/unity/ML-Agents-Pyramids

Just select your model from the dropdown!

## Keep Learning, Stay Awesome 🤗